In [1]:
import os

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn

In [3]:
# read the training data
data = pd.read_csv('train.csv')

In [4]:
# clean the data
data['Sex'] =  data['Sex'].apply(lambda s: 1 if s == 'male' else 0)
data = data.fillna(0)
dataset_X = data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']]
X = dataset_X.as_matrix()

In [5]:
# one-hot encoding the label
data['Deceased'] = data['Survived'].apply(lambda s: int(not s))
dataset_Y = data[['Deceased', 'Survived']]
Y = dataset_Y.as_matrix()

In [6]:
# arguments that can be set in command line
tf.app.flags.DEFINE_integer('epochs', 10, 'Training epochs')
FLAGS = tf.app.flags.FLAGS

In [7]:
ckpt_dir = './ckpt_dir2'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

## TF Learn encapsulation

In [8]:
## Build the Graph
# input
n_features = X.shape[1]
input = tflearn.input_data([None, n_features])
# variables & forward propagation
network = tflearn.layers.fully_connected(input, 10, activation='relu')
network = tflearn.layers.fully_connected(network, 10, activation='relu')
y_pred = tflearn.layers.fully_connected(network, 2, activation='softmax')
# backpropagation
net = tflearn.regression(y_pred, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)
# generate a DNN class object
model = tflearn.DNN(net)

In [9]:
# restore model if there is a checkpoint
if os.path.isfile(os.path.join(ckpt_dir, 'model.ckpt')):
    model.load(os.path.join(ckpt_dir, 'model.ckpt'))

In [10]:
## training process -- just 1 line of code!!
model.fit(X, Y, validation_set=0.1, n_epoch=FLAGS.epochs)

Training Step: 129  | total loss: 0.61181 | time: 0.055s
| Adam | epoch: 010 | loss: 0.61181 -- iter: 768/801
Training Step: 130  | total loss: 0.60296 | time: 1.068s
| Adam | epoch: 010 | loss: 0.60296 | val_loss: 0.69913 -- iter: 801/801
--


In [13]:
# save the trained model
model.save(os.path.join(ckpt_dir, 'model.ckpt'))

INFO:tensorflow:/Users/riyueyoutu/Desktop/USC/deep learning/ckpt_dir2/model.ckpt is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
# evaluate the trained model
metric = model.evaluate(X, Y)
print('Accuracy on train set: %.9f' % metric[0])

Accuracy on train set: 0.698092032


In [14]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='./ckpt_dir2/model.ckpt', tensor_name='', all_tensors=True)

tensor_name:  FullyConnected/W/Adam
[[  1.17114877e-11  -6.07019896e-03  -9.79027152e-03  -1.49413561e-02
    0.00000000e+00   7.58151931e-04   1.22069651e-02  -2.71024392e-03
   -7.46967969e-03  -3.57267982e-03]
 [  0.00000000e+00  -8.33228137e-03   2.99073867e-02   2.48912852e-02
    0.00000000e+00  -1.71193760e-03  -4.98202220e-02   1.67538971e-02
    1.28119001e-02   2.00938750e-02]
 [  1.88262218e-11  -1.38902466e-03  -5.96404495e-03  -5.34004904e-03
    0.00000000e+00  -1.86535192e-03  -1.09972607e-04   1.21573603e-03
   -5.27102640e-03   1.60136912e-03]
 [  0.00000000e+00  -7.20485905e-03  -1.61009165e-03  -1.10693164e-02
    0.00000000e+00   9.33304662e-04   7.55770225e-03  -5.46230283e-03
   -2.78497091e-03  -7.69537687e-03]
 [  0.00000000e+00   2.31031678e-03   3.60049680e-03   4.78247926e-03
    0.00000000e+00  -1.82856122e-04  -4.66477778e-03   1.11401314e-03
    1.96933793e-03   1.21880986e-03]
 [  0.00000000e+00   5.55485748e-02   3.06830704e-02  -1.77917909e-02
    0.000